In [ ]:
import torch
import os
import yaml
import matplotlib.pyplot as plt
import numpy as np
import time

from tqdm import tqdm
from torch_geometric.data import Batch, Data
from pymatgen.core.sites import PeriodicSite
from pymatgen.io.ase import AseAtomsAdaptor
from ase import neighborlist as nbl
from ase import Atoms
from ase.calculators.vasp import VaspChargeDensity

from ocpmodels.common import logger
from ocpmodels.common.registry import registry
from ocpmodels.common.utils import setup_logging
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.datasets import data_list_collater

import cdm.models
from cdm.charge_trainer import ChargeTrainer
from cdm.chg_utils import ProbeGraphAdder

setup_logging()

In [ ]:
def make_parity_plot(x, y, LOG):
    plt.scatter(x, y, 
            color='blue', 
            alpha = 0.1,
            s=1.5,
            #label='Predictions',
          )

    plt.gcf().set_dpi(200)
    plt.axis('square')

    if LOG:
        plt.gca().set_xscale('log')
        plt.gca().set_yscale('log')

    plt.plot([0, np.max(x)], [0, np.max(x)], label='Parity line', color='red')
    plt.xlabel('Ground truth electron density\nelectrons per cubic Angstrom')
    plt.ylabel('Predicted electron density\nelectrons per cubic Angstrom')
    plt.xlim([1e-10, np.max(x)])
    plt.ylim([1e-10, np.max(x)])
    plt.show()

In [ ]:
if torch.cuda.is_available():
    print("True")
else:
    print("False")
    torch.set_num_threads(8)

In [ ]:
file = '../wandb/run-20220721_051246-ls0wfq3n/files/config.yaml'

with open(file) as stream:
    config = yaml.safe_load(stream)

for key, value in config.items():
    print(key, value)

In [ ]:
model_config = {
    'name': 'charge_model',
    'num_interactions': 5,
    'atom_channels': 128,
    'probe_channels': 128,
    
    'atom_model_config': {
        'name': 'schnet_charge',
        'num_filters':64,
        'num_gaussians':64,
        'cutoff':5,
    },
    
    'probe_model_config': {
        'name': 'schnet_charge',
        'num_filters':64,
        'num_gaussians':128,
        'cutoff':4,
    },
}

model = cdm.models.ChargeModel(**model_config)

In [ ]:
path = '../runs/checkpoints/2022-08-16-13-56-16-Quals Run 2/checkpoint.pt'
state_dict = torch.load(path)['state_dict']

sd = {}

for x in state_dict.items():
    #print(x)
    #print(x[0][7:])
    #print(x[1])
    sd[x[0][7:]] = x[1]

model.load_state_dict(sd)

In [ ]:
model.to('cuda')

In [ ]:
a2g = AtomsToGraphs(
        max_neigh = 100,
        radius = 6,
        r_energy = False,
        r_forces = False,
        r_distances = False,
        r_fixed = False,
    )

pga = ProbeGraphAdder(num_probes = 100, cutoff = 3,
                      mode = 'slice', slice_start = 0,
                     stride = 1, implementation = 'RGPBC')

In [ ]:
path = '../../shared-scratch/ethan/density/1k_sample/val/random1005744_130/CHGCAR'

vcd = VaspChargeDensity(path) 
atoms = vcd.atoms[-1]
dens = vcd.chg[-1]
data_object = a2g.convert(atoms)
data_object.charge_density = dens

print(atoms)

In [ ]:
data_object.probe_data = 0
slice0 = pga(data_object.to('cpu'),
             num_probes = 500, 
             mode='all',
             use_tqdm = True,
             stride = 2)
batch = data_list_collater([slice0])
batch.probe_data = Batch.from_data_list([slice0.probe_data])

batch = [batch]

In [ ]:
print(batch[0])

In [ ]:
t1 = time.time()
pred = model(batch[0].to('cuda')).detach().cpu().numpy()
t2 = time.time()

true = batch[0].probe_data.target.detach().cpu().numpy()


print(len(pred))
print(len(true))

print(t2 - t1)

In [ ]:
make_parity_plot(true, pred, LOG=False)
make_parity_plot(true, pred, LOG=True)

In [ ]:
print(np.mean(np.abs((true - pred))).item())